<a href="https://colab.research.google.com/github/wander-asb/US_Accidents/blob/main/Us.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Leitura de arquivos

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('Inicializando com Spark').getOrCreate()

In [ ]:
spark

In [6]:
from google.colab                import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
! mkdir ~/.kaggle

In [8]:
! cp /content/gdrive/MyDrive/kaggle_api/kaggle.json ~/.kaggle/kaggle.json

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
! kaggle datasets download -d sobhanmoosavi/us-accidents

100% 652M/653M [00:33<00:00, 25.5MB/s]
100% 653M/653M [00:33<00:00, 20.4MB/s]


In [11]:
! unzip /content/us-accidents.zip

Archive:  /content/us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [12]:
data_dir = '/content/US_Accidents_March23.csv'

## Bibliotecas

In [48]:
from pyspark.sql.functions import *

## Leitura de dados

### Features:

ID: Um identificador exclusivo para cada acidente.

Source: A fonte que relatou o acidente, como a polícia, operador de transporte público ou usuário do Waze.

TMC: Código de Gerenciamento de Mensagens de Trânsito (TMC) é um código que descreve o evento de trânsito.

Severity: A gravidade estimada do acidente, classificada em uma escala de 1 a 4, sendo 1 o menos grave e 4 o mais grave.

Start_Time: A data e hora em que o acidente ocorreu.

End_Time: A data e hora em que o acidente foi encerrado.

Start_Lat: A latitude do local do acidente.

Start_Lng: A longitude do local do acidente.

End_Lat: A latitude do fim do local do acidente.

End_Lng: A longitude do fim do local do acidente.

Distance(mi): A distância do acidente em milhas.

Description: Uma descrição textual do acidente.

Number: O número da estrada onde o acidente ocorreu.

Street: O nome da rua onde o acidente ocorreu.

Side: O lado da rua onde o acidente ocorreu (esquerda ou direita).

City: A cidade onde o acidente ocorreu.

County: O condado onde o acidente ocorreu.

State: O estado onde o acidente ocorreu.

Zipcode: O código postal da área onde o acidente ocorreu.

Country: O país onde o acidente ocorreu (sempre "US" para este conjunto de dados).

Timezone: O fuso horário em que o acidente ocorreu.

Airport_Code: O código do aeroporto próximo ao local do acidente.

Weather_Timestamp: A data e hora em que as informações meteorológicas foram registradas.

Temperature(F): A temperatura em Fahrenheit.

Wind_Chill(F): A sensação térmica em Fahrenheit.

Humidity(%): A umidade relativa do ar em porcentagem.

Pressure(in): A pressão atmosférica em polegadas de mercúrio.

Visibility(mi): A visibilidade em milhas.

Wind_Direction: A direção do vento.

Wind_Speed(mph): A velocidade do vento em milhas por hora.

Precipitation(in): A quantidade de precipitação em polegadas.

Weather_Condition: A condição climática no momento do acidente.

Amenity: Um indicador se existe ou não uma comodidade (como restaurante, posto de gasolina, etc.) próximo ao local do acidente.

Bump: Um indicador se existe ou não uma irregularidade na estrada próxima ao local do acidente.

Crossing: Um indicador se existe ou não uma travessia de pedestres próxima ao local do acidente.

Give_Way: Um indicador se existe ou não uma placa de "ceda a passagem" próxima ao local do acidente.

Junction: Um indicador se existe ou não um cruzamento próximo ao local do acidente.

No_Exit: Um indicador se existe ou não uma saída próxima ao local do acidente.

Railway: Um indicador se existe ou não uma ferrovia próxima ao local do acidente.

Roundabout: Um indicador se existe ou não uma rotatória próxima ao local do acidente.

Station: Um indicador se existe ou não uma estação (como de ônibus ou de trem) próxima ao local do acidente.

Stop: Um indicador se existe ou não uma parada próxima ao local do acidente.

Traffic_Calming: Um indicador se existe ou não um dispositivo de controle de tráfego (como quebra-molas) próximo ao local do acidente.

Traffic_Signal: Um indicador se existe ou não um semáforo próximo ao local do acidente.

Turning_Loop: Um indicador se existe ou não uma alça de conversão próxima ao local do acidente.

Sunrise_Sunset: Indica se o acidente ocorreu durante o nascer ou pôr do sol.

Civil_Twilight: Indica se o acidente ocorreu durante o crepúsculo civil (luz do dia).

Nautical_Twilight: Indica se o acidente ocorreu durante o crepúsculo náutico (luz reduzida).

Astronomical_Twilight: Indica se o acidente ocorreu durante o crepúsculo astronômico (escuridão total).

In [53]:
us_accident =  spark.read.csv(data_dir, header=True, inferSchema=True)
us_accident.show(5)

+---+-------+--------+-------------------+-------------------+-----------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID| Source|Severity|         Start_Time|           End_Time|        Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Ameni

In [34]:
us_accident.count()

7728394

In [35]:
us_accident.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: string (nullable = true)
 |-- End_Time: string (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- Visibility

## Análise Exploratória de Dados

## Análise de Estados e cidades

In [52]:
#Indicador dos Estados que mais ocorrem acidentes nos Estados Unidos
us_accident.groupby('State').agg(count('ID').alias('Qtd de acidentes')).sort(desc('Qtd de acidentes')).show()

+-----+----------------+
|State|Qtd de acidentes|
+-----+----------------+
|   CA|         1741433|
|   FL|          880192|
|   TX|          582837|
|   SC|          382557|
|   NY|          347960|
|   NC|          338199|
|   VA|          303301|
|   PA|          296620|
|   MN|          192084|
|   OR|          179660|
|   AZ|          170609|
|   GA|          169234|
|   IL|          168958|
|   TN|          167388|
|   MI|          162191|
|   LA|          149701|
|   NJ|          140719|
|   MD|          140417|
|   OH|          118115|
|   WA|          108221|
+-----+----------------+
only showing top 20 rows



In [72]:
#Avaliamos as 10 cidades que mais ocorreram relatos de acidentes de acordo com a condição climática
df_cidades = us_accident.where(
    col('State')=='CA'
    ).withColumn('Date', to_date(col('Start_time'))).select('Date','Weather_Condition','City')\
      .groupby('City','Weather_Condition').agg(count('Date').alias('Qtde de acidentes'))

#Ao definir o tipo de condição climárica por cidade e apresentando seus respectivos dados quantitativos.
      #Avalia-se então, qual cidade apresenta maior número de acidentes e qual condição climática torna-se causa de preocupação?